# Tracking the Sun 2019 with States

OEDI Berkeley Lab’s *Tracking the Sun* report series is dedicated to summarizing installed prices and other trends among grid-connected, distributed solar photovoltaic (PV) systems in the United States. For more information, please refer to https://emp.lbl.gov/tracking-the-sun/

Geospatial data in data lake can assist on PV system price analysis and visualization, and discover trends or patterns of PV system installation price across the whole country. 

We assume your OEDI data lake has already been deployed. In this example, the deployed database in data lake is `oedi_database_test`.

* Tracking the Sun data table is `lbnl_tracking_the_sun_2019`
* The geospatial data table is `oedi_sera_gis_census_state_2010`

In [1]:
import folium
from branca import colormap
from branca.element import Template, MacroElement

from oedi.AWS.utils.glue import OEDIGlue
from oedi.AWS.utils.athena import OEDIAthena

In [2]:
glue = OEDIGlue()
athena = OEDIAthena(staging_location="s3://nrel-tests/tracking-the-sun", region_name="us-west-2")

In [3]:
database_name = "oedi_database_test"
tts_table_name = "lbnl_tracking_the_sun_2019"
state_table_name = "oedi_sera_gis_census_state_2010"

In [4]:
# Query from two tables in data lake
query_string = f"""
    WITH 
        t1 As (
            SELECT state, COUNT(*) AS count
            FROM {database_name}.{tts_table_name}
            GROUP BY state
        ),
        t2 AS (
            SELECT state_usps, census_state_geometry_500k
            FROM {database_name}.{state_table_name}
        )
    SELECT state, count, census_state_geometry_500k
    FROM t1 INNER JOIN t2
    ON t1.state=t2.state_usps
"""
pv_state = athena.run_query(query_string, geometry="census_state_geometry_500k")

In [5]:
pv_state

,state,count,census_state_geometry_500k
0,FL,6270,"MULTIPOLYGON (((-85.15641 29.67963, -85.13740 ..."
1,NJ,129049,"MULTIPOLYGON (((-75.52684 39.65571, -75.52634 ..."
2,KS,316,"MULTIPOLYGON (((-94.61808 36.99813, -94.62522 ..."
3,WA,5303,"MULTIPOLYGON (((-123.09055 49.00198, -123.0353..."
4,RI,994,"MULTIPOLYGON (((-71.28157 41.64821, -71.27817 ..."
5,CO,93930,"MULTIPOLYGON (((-102.04224 36.99308, -102.0545..."
6,IL,2926,"MULTIPOLYGON (((-87.53233 39.99778, -87.53254 ..."
7,WI,4039,"MULTIPOLYGON (((-90.45525 47.02400, -90.45713 ..."
8,OR,4965,"MULTIPOLYGON (((-117.22007 44.30138, -117.2224..."
9,MO,15228,"MULTIPOLYGON (((-89.53910 36.49820, -89.53452 ..."


In [6]:
# Display state pv system numbers on map
state_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="OpenStreetMap")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colors = ["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"]
bins = [0, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
colorscale = colormap.StepColormap(
    colors=colors,
    index=bins,
    vmin=0,
    vmax=1000000
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=pv_state.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(state_map)

legend = MacroElement()
with open("Tracking-the-Sun-with-State-Legend.html") as f:
    template = f.read()
legend._template = Template(template)
state_map.get_root().add_child(legend)

state_map

From the map, we can check the geospatial distribution of PV system installation across the United States.